--- Day 3: Gear Ratios ---

You and the Elf eventually reach a gondola lift station; he says the gondola lift will take you up to the water source, but this is as far as he can bring you. You go inside.

It doesn't take long to find the gondolas, but there seems to be a problem: they're not moving.

"Aaah!"

You turn around to see a slightly-greasy Elf with a wrench and a look of surprise. "Sorry, I wasn't expecting anyone! The gondola lift isn't working right now; it'll still be a while before I can fix it." You offer to help.

The engineer explains that an engine part seems to be missing from the engine, but nobody can figure out which one. If you can add up all the part numbers in the engine schematic, it should be easy to work out which part is missing.

The engine schematic (your puzzle input) consists of a visual representation of the engine. There are lots of numbers and symbols you don't really understand, but apparently any number adjacent to a symbol, even diagonally, is a "part number" and should be included in your sum. (Periods (.) do not count as a symbol.)

Here is an example engine schematic:

467..114..
...*......
..35..633.
......#...
617*......
.....+.58.
..592.....
......755.
...$.*....
.664.598..

In this schematic, two numbers are not part numbers because they are not adjacent to a symbol: 114 (top right) and 58 (middle right). Every other number is adjacent to a symbol and so is a part number; their sum is 4361.

Of course, the actual engine schematic is much larger. What is the sum of all of the part numbers in the engine schematic?


In [91]:
! pip install termcolor
from termcolor import colored

  Obtaining dependency information for termcolor from https://files.pythonhosted.org/packages/d9/5f/8c716e47b3a50cbd7c146f45881e11d9414def768b7cd9c5e6650ec2a80a/termcolor-2.4.0-py3-none-any.whl.metadata
Using cached termcolor-2.4.0-py3-none-any.whl (7.7 kB)



[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: C:\Users\dkwar\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [239]:
class Point:
    def __init__(self, row, column):
        self.row = row
        self.column = column
    
    def __eq__(self, other):
        return self.row == other.row and self.column == other.column
    
    def __hash__(self):
        return hash((self.row, self.column))
    
    def __str__(self):
        return "Point: row: " + str(self.row) + " column: " + str(self.column)

class Gear:
    def __init__(self):
        self.__points = {}
        self.__start_row = None
        self.__symbols = {}

    def get_start_row(self):
        return self.__start_row

    def add_value(self, row, column, value):
        if self.__start_row is None:
            self.__start_row = row

        if self.__points.get(Point(row, column), None) is None:
            if row == self.__start_row:
                self.__points[Point(row, column)] = int(value)

    def has_any_value(self):
        return len(self.__points) > 0
    
    def add_symbol(self, row, col, symbol):
        if self.__symbols.get(Point(row, col), None) is None:
            self.__symbols[Point(row, col)] = symbol

    def has_any_symbol(self):
        return len(self.__symbols) > 0
    
    def read_gear(self):
        gear_value = ""
        for point in self.__points.keys():
            if point.row == self.__start_row:
                gear_value += str(self.__points[point])
        return gear_value
    
    def is_point_in_gear(self, point):
        return point in self.__points
    
    def is_symbol_point_in_gear(self, point, symbol):
        return point in self.__symbols and self.__symbols[point] == symbol

    def __str__(self):
        return "Gear: " + self.read_gear() + " has_symbol: " + str(self.has_any_symbol()) + " is_valid: " + str(self.is_valid())

class GearRatio:
    def __init__(self, gear1=None, gear2=None):
        self.gear1 = gear1
        self.gear2 = gear2
    
    def calculate(self):
        return int(self.gear1.read_gear()) * int(self.gear2.read_gear())
    
    def __str__(self):
        return "GearRatio: " + str(self.gear1.read_gear()) + " * " + str(self.gear2.read_gear())

class GearBox:
    def __init__(self, gears_schematic, numbers_from=0, numbers_to=10, fillers=['.'], special_symbols=["*"]):
        self.gears_schematic_array = self.create_gears_array(gears_schematic)
        self.valid_gears = []
        self.valid_speciag_gear_pairs = []
        self.invalid_gears = []
        self.used_symbols = {}
    
        self.numbers = [str(i) for i in range(numbers_from, numbers_to)]
        self.special_symbols = special_symbols
        self.fillers = fillers

        print("Numbers: " + "".join(self.numbers))

    def create_gears_array(self, gears_schematic):
        gears_schematic_array = [list(line) for line in gears_schematic.split('\n')]
        return gears_schematic_array

    def check_gear_array(self):
        for row_idx, row in enumerate(self.gears_schematic_array):
            gear = Gear()
            for col_idx, column_value in enumerate(row):
                if column_value not in self.numbers or col_idx == len(row) - 1:
                    if gear.has_any_value() is True:
                        if gear.has_any_symbol() is True:
                            self.valid_gears.append(gear)
                        else:
                            self.invalid_gears.append(gear)
                    gear = Gear()
                else:
                    gear.add_value(row_idx, col_idx, column_value)
                    gear = self.__look_at_adjustent_fields(row_idx, col_idx, gear)        
      
    def check_valid_gears(self, special_symbol='*'):
        checked_gears = set()  # Używamy zbioru do śledzenia sprawdzonych gear ratios
        for point in self.used_symbols.get(special_symbol, []):
            for gear1 in self.valid_gears:
                if gear1 in checked_gears:
                    continue
                if gear1.is_symbol_point_in_gear(point, special_symbol):
                    for gear2 in self.valid_gears:
                        if gear2 in checked_gears or gear1 == gear2:
                            continue
                        if gear2.is_symbol_point_in_gear(point, special_symbol):
                            self.valid_speciag_gear_pairs.append(GearRatio(gear1, gear2))
                            checked_gears.update([gear1, gear2])
                            break

    def __look_at_adjustent_fields(self, row, col, gear):
        rows, cols = len(self.gears_schematic_array), len(self.gears_schematic_array[0])
        directions = [(0, 1), (1, 0), (0, -1), (-1, 0)]
        diagonal_directions = [(1, 1), (1, -1), (-1, 1), (-1, -1)]
        directions.extend(diagonal_directions)

        for i, j in directions:
            gear = self.__handle_look(row + i, col + j, rows, cols, gear) or gear
        return gear

    def __handle_look(self, row, col, rows, cols, gear):
        if not (0 <= row < rows and 0 <= col < cols):
            return gear
        
        cell_value = self.gears_schematic_array[row][col]

        if cell_value in self.fillers:
            return gear
        
        if cell_value in self.numbers:
            gear.add_value(row, col, cell_value)
        else:
            gear.add_symbol(row, col, cell_value)
            if cell_value not in self.used_symbols:
                self.used_symbols[cell_value] = []
            self.used_symbols[cell_value].append(Point(row, col))
              
    def print_valid_gears(self):
        for gear in self.valid_gears:
            print(gear)
    
    def print_invalid_gears(self):
        for gear in self.invalid_gears:
            print(gear)

    def __sum_gears(self, gears):
        sum = 0
        for gear in gears:
            sum += int(gear.read_gear())
        return sum
    
    def print_sum_valid_gears(self):
        print("Valid gears:")
        print("Sum of valid gears: " + str(self.__sum_gears(self.valid_gears)))

    def print_sum_invalid_gears(self):
        print("Invalid gears:")
        print("Sum of invalid gears: " + str(self.__sum_gears(self.invalid_gears)))

    def print_rows_and_cols_count(self):
        rows_count = len(self.gears_schematic_array)
        cols_count = len(self.gears_schematic_array[0])
        print("Rows: " + str(rows_count) + " Cols: " + str(cols_count))

    def print_colored_gears_array(self, rows_from=None, rows_to=None, highlight_row=None, space_between_gears=" "):
        if rows_from is None or rows_from < 0:
            rows_from = 0
        if rows_to is None or rows_to > len(self.gears_schematic_array):
            rows_to = len(self.gears_schematic_array)

        array = self.gears_schematic_array[rows_from:rows_to]
        for local_row_idx, row in enumerate(array):
            colored_row = ""
            global_row_idx = local_row_idx + rows_from
            for col_idx, col in enumerate(row):
                point = Point(global_row_idx, col_idx)
                char_to_print = str(col)
                if self.__is_point_in_gear_ratios(point, self.valid_speciag_gear_pairs):
                    colored_row += colored(char_to_print, 'yellow')
                elif self.__is_point_in_gears(point, self.valid_gears):
                    colored_row += colored(char_to_print, 'green')
                elif self.__is_point_in_gears(point, self.invalid_gears):
                    colored_row += colored(char_to_print, 'red')
                elif self.used_symbols.get(char_to_print) is not None and point in self.used_symbols[char_to_print]:
                    if  char_to_print in self.special_symbols:
                        colored_row += colored(char_to_print, 'yellow')
                    else:
                        colored_row += colored(char_to_print, 'blue')
                else:
                    colored_row += char_to_print
                colored_row += space_between_gears
            if highlight_row is not None and global_row_idx == highlight_row:
                colored_row += colored('←', 'yellow')
            print(colored_row)

    def __is_point_in_gears(self, point, gears):
        for gear in gears:
            if gear.is_point_in_gear(point):
                return True
        return False
    
    def __is_point_in_gear_ratios(self, point, gear_ratios):
        for gear_ratio in gear_ratios:
            if gear_ratio.gear1.is_point_in_gear(point) or gear_ratio.gear2.is_point_in_gear(point):
                return True
        return False
    
    def print_valid_special_gear_pairs(self):
        print("\nValid special gear pairs: ")
        for gear_ratio in self.valid_speciag_gear_pairs:
            print(gear_ratio)
    
    def print_rows_with_gears(self, row_index, space_between_gears=" "):
        self.print_colored_gears_array(row_index-1, row_index+2, row_index, space_between_gears)

        print("\nGears with starting row {}: ".format(row_index))
        for gear in self.valid_gears + self.invalid_gears:
            if gear.get_start_row() == row_index:
                print(gear)

    def print_all_rows_with_gears(self, space_between_gears=" "):
        rows = len(self.gears_schematic_array)
        for row_index in range(rows):
            print(f"Row {row_index}:")
            self.print_rows_with_gears(row_index, space_between_gears)
            print("\n")

    def __sum_gear_ratios(self):
        sum = 0
        for gear_ratio in self.valid_speciag_gear_pairs:
            sum += gear_ratio.calculate()
        return sum
    
    def print_sum_gear_ratios(self):
        print("Sum of gear ratios: " + str(self.__sum_gear_ratios()))


In [240]:
linesText = """467..114..
...*......
..35..633.
......#...
617*......
.....+.58.
..592.....
......755.
...$.*....
.664.598.."""

gear_box = GearBox(linesText)
#gear_box.print_rows_and_cols_count()
gear_box.check_gear_array()
gear_box.check_valid_gears(special_symbol='*')

gear_box.print_sum_gear_ratios()
gear_box.print_valid_special_gear_pairs()
gear_box.print_sum_valid_gears()
#gear_box.print_valid_gears()
gear_box.print_sum_invalid_gears()
#gear_box.print_invalid_gears()
gear_box.print_colored_gears_array(space_between_gears="  ")
#gear_box.print_all_rows_with_gears(space_between_gears=" ")


Numbers: 0123456789
Sum of gear ratios: 467835

Valid special gear pairs: 
GearRatio: 467 * 35
GearRatio: 755 * 598
Valid gears:
Sum of valid gears: 4361
Invalid gears:
Sum of invalid gears: 172
4  6  7  .  .  1  1  4  .  .  
.  .  .  *  .  .  .  .  .  .  
.  .  3  5  .  .  6  3  3  .  
.  .  .  .  .  .  #  .  .  .  
6  1  7  *  .  .  .  .  .  .  
.  .  .  .  .  +  .  5  8  .  
.  .  5  9  2  .  .  .  .  .  
.  .  .  .  .  .  7  5  5  .  
.  .  .  $  .  *  .  .  .  .  
.  6  6  4  .  5  9  8  .  .  


In [242]:
with open('input.txt', 'r') as file:
    linesText = file.read()

gear_box = GearBox(linesText, numbers_from=0, numbers_to=10, fillers=['.'])
gear_box.check_gear_array()
gear_box.check_valid_gears(special_symbol='*')

gear_box.print_sum_gear_ratios()
#gear_box.print_valid_special_gear_pairs()
gear_box.print_sum_valid_gears()
#gear_box.print_valid_gears()
gear_box.print_sum_invalid_gears()
#gear_box.print_invalid_gears()
gear_box.print_colored_gears_array(space_between_gears="")
#gear_box.print_all_rows_with_gears(space_between_gears="")


Numbers: 0123456789
Sum of gear ratios: 79026871
Valid gears:
Sum of valid gears: 527364
Invalid gears:
Sum of invalid gears: 76022
...................15....904...........850.................329...................13....................................871....816....697....
...........53.497........................%....906...610.......*.............735#..&...*......558...68...............68..*......&....*.......
..........*....$....................132.........*..........844....875................350............*...............*..336.364...649........
.......726.......341..................*...186...358..................*244........57.......@.........738......*.....663.................584..
.............952.*......33......660..704............949......................518*....234.967....551........971..&.......................*...
.......738...*....222......................706.......*..825.............474%...........*...........*.405.........779..............542...405.
.74.........366.......

Your puzzle answer was 79026871.

Both parts of this puzzle are complete! They provide two gold stars: **